In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import os
import re
import shutil
import time
import zipfile

from pathlib import Path
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import secrets
import util

def main():
    chrome_options = webdriver.ChromeOptions()
    
    prefs = {
        'download.default_directory': secrets.acorn_download_dir,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True
    }
    chrome_options.add_experimental_option('prefs', prefs)
    
    driver = webdriver.Chrome(options=chrome_options)

    wait = WebDriverWait(driver, 10)

    driver.get(secrets.acorn_url)

    # Get rid of the cookies popup
    #iframe = WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'consent-iframe')))

    iframe_element = wait.until(EC.element_to_be_clickable((By.ID, "consent-iframe")))
    driver.switch_to.frame(iframe_element)
    print(f'{iframe_element=}')

    iframe_element2 = wait.until(EC.element_to_be_clickable((By.ID, "ccpa-iframe")))
    driver.switch_to.frame(iframe_element2)
    print(f'{iframe_element2=}')

    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'accept_all_cookies_button')))
    print(f'{accept_button=}')
    accept_button.click()

    driver.switch_to.default_content()
    
    year_folder = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{secrets.year}')]")))
    year_folder.click()
    
    download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@aria-label='Download']")))
    download_button.click()
    time.sleep(5)
    print('Done!')

    zip_file_path = Path(secrets.acorn_download_dir) / f'{secrets.year}.zip'
    extract_to_directory = secrets.acorn_download_dir

    time.sleep(2)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_directory)

    util.move_from_download_to_staging(secrets.acorn_input_regex, secrets.acorn_output_regex,
                                       secrets.acorn_download_dir, 
                                       secrets.acorn_staging_dir,
                                       secrets.acorn_account_dirs, 'Acorn',
                                       secrets.acorn_allow_patterns)
    util.copy_to_destination(secrets.acorn_account_dirs,
                            secrets.acorn_staging_dir,
                            secrets.destination_dir)

    # cleanup
    shutil.rmtree(secrets.acorn_staging_dir)
    shutil.rmtree(secrets.acorn_download_dir)


main()

iframe_element=<selenium.webdriver.remote.webelement.WebElement (session="caf0c6b5c3a043e2dcb9b7b885f89f6b", element="B739E5653EDAFB7F9FAE8FC8EF160799_element_30")>
iframe_element2=<selenium.webdriver.remote.webelement.WebElement (session="caf0c6b5c3a043e2dcb9b7b885f89f6b", element="2893D2DF7463F24072A7299C405C4F91_element_32")>
accept_button=<selenium.webdriver.remote.webelement.WebElement (session="caf0c6b5c3a043e2dcb9b7b885f89f6b", element="8ECB8E35A246622C8104649CDF02A679_element_33")>
Done!
account_dirs={'Acorn': 'Property/Encanto/Property Management/Statements'}
Copying account Acorn
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Encanto/Property Management/Statements/2023_04.pdf
